In [ ]:
import os
import yaml
from transformers import AutoModelForCausalLM
from huggingface_hub import HfApi, HfFolder, Repository
from huggingface_hub.utils._errors import RepositoryNotFoundError

# Load config file into dictionary
with open('./config/instructor_config.yaml', 'r') as file:
    config = yaml.safe_load(file)
print(config)

hf_key_path = config.get("hf_key_path")
if hf_key_path is not None:
    with open(hf_key_path, "r") as file:
        hf_auth = file.read().strip()
else:
    hf_auth = None
    
model_dir = os.path.join(config["training_params"]["output_dir"], config["checkpoint_name"])
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map='auto')

In [ ]:
# Initialize API
api = HfApi()

# Function to check if the repo exists
def repo_exists(repo_id, token):
    try:
        api.repo_info(repo_id=repo_id, token=token)
        return True
    except RepositoryNotFoundError:
        return False

# Check if the repository exists
if repo_exists(config["model_repo_id"], hf_auth):
    print(f"Not pushing model as repository {config['model_repo_id']} already exists.")
else:
    if config.get("push_to_hub") == True:
        model.push_to_hub(config["model_repo_id"], use_auth_token=hf_auth)
    else:
        print(f"Not pushing model as config 'push_to_hub' parameter not set to True")